In [1]:
import os
os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

import sys
sys.path.append('..')

from datetime import datetime

import numpy as np
import pandas as pd

import brasa

In [6]:
import pyarrow
import pyarrow.compute as pc

def get_indexes_names() -> list:
    tb = brasa.get_dataset("b3-indexes-composition").scanner(columns=["refdate"]).to_table()
    max_date = pyarrow.compute.max(tb.column("refdate"))
    df = brasa.get_dataset("b3-indexes-composition")\
        .filter(pc.field("refdate") == max_date)\
        .scanner(columns=["indexes"])\
        .to_table().to_pandas()
    return list(df.indexes.unique())

get_indexes_names()

['BDRX',
 'GPTW',
 'IBOV',
 'IBRA',
 'IBXL',
 'IBXX',
 'IGCT',
 'IGCX',
 'IGNM',
 'ITAG',
 'IVBX',
 'SMLL',
 'AGFS',
 'ICON',
 'IDIV',
 'IFNC',
 'INDX',
 'ISEE',
 'IDVR',
 'IEEX',
 'UTIL',
 'IMOB',
 'ICO2',
 'MLCX',
 'IBSD',
 'IMAT',
 'IFIX',
 'IFIL']

In [9]:
from brasa import download_marketdata

names = get_indexes_names()
download_marketdata("b3-indexes-theoretical-portfolio", index="IDVR")
# download_marketdata("b3-indexes-composition")

In [10]:
from brasa import process_marketdata

process_marketdata("b3-indexes-theoretical-portfolio")

Processing b3-indexes-theoretical-portfolio   0/1  |    | Elapsed Time: 0:00:00
Processing b3-indexes-theoretical-portfolio   1/1  |####| Elapsed Time: 0:00:00


{'index': 'IDVR'} 'part'


In [11]:
from brasa.downloaders.downloaders import B3PagedURLEncodedDownloader, B3URLEncodedDownloader
url = "https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetTheoricalPortfolio"
kwargs = {
    "index": "IDVR",
    "language": "pt-br",
    "pageNumber": 1,
    "pageSize": 999,
}

dnd = B3URLEncodedDownloader(url, True, **kwargs)
fp = dnd.download()
import json

fp.seek(0)
json.load(fp)

{'page': {'pageNumber': 1,
  'pageSize': 999,
  'totalRecords': None,
  'totalPages': None},
 'header': None,
 'results': None}

In [42]:
fp.seek(0)
obj = json.load(fp)

In [43]:
df = pd.DataFrame(obj["results"])
df["part"] = pd.to_numeric(df["part"].str.replace(".", "").str.replace(",", "."))
df["theoricalQty"] = pd.to_numeric(df["theoricalQty"].str.replace(".", "").str.replace(",", "."))
df["total_theorical_qty"] = obj["header"]["theoricalQty"]
df["reductor"] = obj["header"]["reductor"]
df["total_theorical_qty"] = pd.to_numeric(df["total_theorical_qty"].str.replace(".", "").str.replace(",", "."))
df["reductor"] = pd.to_numeric(df["reductor"].str.replace(".", "").str.replace(",", "."))
df

,cod,asset,type,theoricalQty,part,cont,total_theorical_qty,reductor
0,ABEV3,AMBEV S/A,ON,4394245879,2.829,4,102454440383,4.340707e+07
1,ALPA4,ALPARGATAS,PN N1,208965950,0.086,3,102454440383,4.340707e+07
2,ALSO3,ALIANSCSONAE,ON NM,532616595,0.568,2,102454440383,4.340707e+07
3,ARZZ3,AREZZO CO,ON NM,62305891,0.213,5,102454440383,4.340707e+07
4,ASAI3,ASSAI,ON NM,1349645897,0.743,6,102454440383,4.340707e+07
...,...,...,...,...,...,...,...,...
95,VIIA3,VIA,ON NM,1582929157,0.095,96,102454440383,4.340707e+07
96,VIVA3,VIVARA S.A.,ON NM,107725671,0.142,98,102454440383,4.340707e+07
97,VIVT3,TELEF BRASIL,ON EJ,423091712,0.819,88,102454440383,4.340707e+07
98,WEGE3,WEG,ON NM,1481593024,2.473,99,102454440383,4.340707e+07


In [4]:
from brasa.downloaders.downloaders import B3PagedURLEncodedDownloader, B3URLEncodedDownloader
url = "https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetStockIndex"
kwargs = {
    "pageNumber": 1,
    "pageSize": 999,
}

dnd = B3URLEncodedDownloader(url, True, **kwargs)
fp = dnd.download()
import json

fp.seek(0)
json.load(fp)

{'page': {'pageNumber': 1,
  'pageSize': 999,
  'totalRecords': 513,
  'totalPages': 1},
 'header': {'update': '2023-12-26',
  'startMonth': 9,
  'endMonth': 12,
  'year': 2023},
 'results': [{'company': '3M',
   'spotlight': 'DRN',
   'code': 'MMMC34',
   'indexes': 'BDRX'},
  {'company': '3R PETROLEUM',
   'spotlight': 'ON   NM',
   'code': 'RRRP3',
   'indexes': 'GPTW,IBOV,IBRA,IBXL,IBXX,IGCT,IGCX,IGNM,ITAG,IVBX,SMLL'},
  {'company': '3TENTOS',
   'spotlight': 'ON   NM',
   'code': 'TTEN3',
   'indexes': 'AGFS,IBRA,ICON,IGCT,IGCX,IGNM,ITAG,SMLL'},
  {'company': 'ABBVIE',
   'spotlight': 'DRN',
   'code': 'ABBV34',
   'indexes': 'BDRX'},
  {'company': 'ABC BRASIL',
   'spotlight': 'PN   N2',
   'code': 'ABCB4',
   'indexes': 'IBRA,IDIV,IFNC,IGCT,IGCX,ITAG,SMLL'},
  {'company': 'ADOBE INC',
   'spotlight': 'DRN',
   'code': 'ADBE34',
   'indexes': 'BDRX'},
  {'company': 'ADVANCED MIC',
   'spotlight': 'DRN',
   'code': 'A1MD34',
   'indexes': 'BDRX'},
  {'company': 'AERIS',
   'spotli

In [5]:
fp.seek(0)
obj = json.load(fp)
df = pd.DataFrame(obj["results"])
df_stock_indexes = df.groupby(["company", "spotlight", "code"]).apply(lambda x: x.indexes.str.split(",").explode()).reset_index()

header = obj["header"]
df_stock_indexes["start_month"] = f"{header['year']}-{str.zfill(str(header['startMonth']), 2)}"
df_stock_indexes["end_month"] = f"{header['year']}-{str.zfill(str(header['endMonth']), 2)}"
df_stock_indexes["index_update_date"] = pd.to_datetime(header["update"])

df_stock_indexes

,company,spotlight,code,level_3,indexes,start_month,end_month,index_update_date
0,3M,DRN,MMMC34,0,BDRX,2023-09,2023-12,2023-12-26
1,3R PETROLEUM,ON NM,RRRP3,1,GPTW,2023-09,2023-12,2023-12-26
2,3R PETROLEUM,ON NM,RRRP3,1,IBOV,2023-09,2023-12,2023-12-26
3,3R PETROLEUM,ON NM,RRRP3,1,IBRA,2023-09,2023-12,2023-12-26
4,3R PETROLEUM,ON NM,RRRP3,1,IBXL,2023-09,2023-12,2023-12-26
...,...,...,...,...,...,...,...,...
2383,ZAMP S.A.,ON NM,ZAMP3,511,IGCX,2023-09,2023-12,2023-12-26
2384,ZAMP S.A.,ON NM,ZAMP3,511,IGNM,2023-09,2023-12,2023-12-26
2385,ZAMP S.A.,ON NM,ZAMP3,511,ITAG,2023-09,2023-12,2023-12-26
2386,ZAMP S.A.,ON NM,ZAMP3,511,SMLL,2023-09,2023-12,2023-12-26


In [5]:
from brasa.engine import retrieve_template, CacheMetadata, CacheManager
import json
import gzip

template_name = "b3-indexes-composition"
template = retrieve_template(template_name)
meta = CacheMetadata(template.id)
meta.extra_key = template.downloader.extra_key
cache = CacheManager()
meta.download_args = dict(index="IBOV")
cache.download_marketdata(meta)

with gzip.open(os.path.join("D:\\brasa", meta.downloaded_files[0])) as f:
        s = f.read()
        obj = json.loads(s)
obj

{'page': {'pageNumber': 1,
  'pageSize': 9999,
  'totalRecords': 513,
  'totalPages': 1},
 'header': {'update': '2023-12-26',
  'startMonth': 9,
  'endMonth': 12,
  'year': 2023},
 'results': [{'company': '3M',
   'spotlight': 'DRN',
   'code': 'MMMC34',
   'indexes': 'BDRX'},
  {'company': '3R PETROLEUM',
   'spotlight': 'ON   NM',
   'code': 'RRRP3',
   'indexes': 'GPTW,IBOV,IBRA,IBXL,IBXX,IGCT,IGCX,IGNM,ITAG,IVBX,SMLL'},
  {'company': '3TENTOS',
   'spotlight': 'ON   NM',
   'code': 'TTEN3',
   'indexes': 'AGFS,IBRA,ICON,IGCT,IGCX,IGNM,ITAG,SMLL'},
  {'company': 'ABBVIE',
   'spotlight': 'DRN',
   'code': 'ABBV34',
   'indexes': 'BDRX'},
  {'company': 'ABC BRASIL',
   'spotlight': 'PN   N2',
   'code': 'ABCB4',
   'indexes': 'IBRA,IDIV,IFNC,IGCT,IGCX,ITAG,SMLL'},
  {'company': 'ADOBE INC',
   'spotlight': 'DRN',
   'code': 'ADBE34',
   'indexes': 'BDRX'},
  {'company': 'ADVANCED MIC',
   'spotlight': 'DRN',
   'code': 'A1MD34',
   'indexes': 'BDRX'},
  {'company': 'AERIS',
   'spotl

In [5]:
import pyarrow
import pyarrow.compute as pc

def get_company_trading_names() -> list:
    tb = brasa.get_dataset("b3-company-details").scanner(columns=["refdate"]).to_table()
    max_date = pyarrow.compute.max(tb.column("refdate"))
    df = brasa.get_dataset("b3-company-details")\
        .filter(pc.field("refdate") == max_date)\
        .scanner(columns=["tradingName"])\
        .to_table().to_pandas()
    return list(df.tradingName.unique())

In [6]:
# from brasa import download_marketdata

# names = get_company_trading_names()
# download_marketdata("b3-cash-dividends", tradingName=names)

In [7]:
from brasa import process_marketdata

process_marketdata("b3-cash-dividends")

Processing b3-cash-dividends   0/312|                   | Elapsed Time: 0:00:00
Processing b3-cash-dividends   1/312|                   | Elapsed Time: 0:00:00
Processing b3-cash-dividends   2/312|                   | Elapsed Time: 0:00:01
Processing b3-cash-dividends   3/312|                   | Elapsed Time: 0:00:02
Processing b3-cash-dividends   4/312|                   | Elapsed Time: 0:00:02
Processing b3-cash-dividends   5/312|                   | Elapsed Time: 0:00:02
Processing b3-cash-dividends   6/312|                   | Elapsed Time: 0:00:03
Processing b3-cash-dividends   7/312|                   | Elapsed Time: 0:00:03
Processing b3-cash-dividends   8/312|                   | Elapsed Time: 0:00:04
Processing b3-cash-dividends   9/312|                   | Elapsed Time: 0:00:04
Processing b3-cash-dividends  10/312|                   | Elapsed Time: 0:00:04
Processing b3-cash-dividends  11/312|                   | Elapsed Time: 0:00:05
Processing b3-cash-dividends  12/312|   